::: {.callout-note}

## Learning Outcomes

- Dissect Singular Value Decomposition (SVD) and use it to calculate principal components.
- Develop a deeper understanding of how to interpret Principal Components Analysis (PCA).
- See applications of PCA to some real-world contexts.
  :::

## Singular Value Decomposition (SVD)

Singular value decomposition (SVD) is an important concept in linear algebra. Since this class requires a linear algebra course (MATH 54, 56 or EECS 16A) as a pre/co-requisite, we assume you have taken or are taking a linear algebra course, so we won't explain SVD in its entirety. In particular, we will go over:

- Why SVD is a valid decomposition of rectangular matrices
- Why PCA is an application of SVD.

We will not go much into the theory and details of SVD. Instead, we will only cover what is needed for a data science interpretation. If you'd like more information, check out [EECS 16B Note 14](https://eecs16b.org/notes/sp23/note14.pdf) or [EECS 16B Note 15](https://eecs16b.org/notes/sp23/note15.pdf).

::: {.callout-tip}

### [Linear Algebra Review] Orthonormality

Orthonormal is a combination of two words: orthogonal and normal.

When we say the columns of a matrix are orthonormal, we say that

1. The columns are all orthogonal to each other (all pairs of columns have a dot product of zero)
2. All columns are unit vectors (the length of each column vector is 1)
<center><img src = "images/orthonormal.png" width="400vw"></center>

Orthonormal matrices have a few important properties

- **Orthonormal inverse**: If an $m \times n$ matrix $Q$ has orthonormal columns, $QQ^T= Iₘ$ and $Q^TQ=Iₙ$.
- **Rotation of coordinates**: the linear transformation represented by an orthonormal matrix is often a rotation (and less often a reflection). We can imagine columns of the matrix as where the unit vectors of the original space will land.
  :::

::: {.callout-tip}

### [Linear Algebra Review] Diagonal Matrices

**Diagonal matrices** are square matrices with non-zero values on the diagonal axis and zeros everywhere else.

Right-multiplied diagonal matrices scale each column up or down by a constant factor. Geometrically, this transformation can be viewed as scaling the coordinate system.

<center><img src = "images/diag_matrix.png" width="600vw"></center>
:::

Singular value decomposition (SVD) describes a matrix $X$'s decomposition into three matrices:
$$ X = U S V^T $$

Let's break down each of these terms one by one.

### $U$

- $U$ is an $n \times d$ matrix: $U \in \mathbb{R}^{n \times d}$.
- Its columns are **orthonormal**
  - $\vec{u_i}^T\vec{u_j} = 0$ for all pairs $i, j$
  - all vectors $\vec{u_i}$ are unit vectors where $|| \vec{u_i} || = 1$ .
- Columns of U are called the **left singular vectors**.
- $UU^T = I_n$ and $U^TU = I_d$.
- We can think of $U$ as a rotation.

<center><img src = "images/u.png" width="600vw"></center>

### $S$

- $S$ is a $d \times d$ matrix: $S \in \mathbb{R}^{d \times d}$.
- The majority of the matrix is zero
- It has $r$ **non-zero** **singular values**, and $r$ is the rank of $X$
- Diagonal values (**singular values** $s_1, s_2, ... s_r$), are **non-negative** ordered from greatest to least: $s_1 \ge s_2 \ge ... \ge s_r > 0$
- We can think of $S$ as scaling.

<center><img src = "images/s.png" width="400vw"></center>

### $V^T$

- $V^T$ is an $d \times d$ matrix: $V \in \mathbb{R}^{d \times d}$.
- Columns of $V$ are orthonormal, so the rows of $V^T$ are orthonormal
- Columns of $V$ are called the **right singular vectors**.
- $VV^T = V^TV = I_d$
- We can think of $V$ as a rotation.

<center><img src = "images/v.png" width="300vw"></center>

<!-- ### SVD: Geometric Perspective

<center><img src = "images/svd.png" width="500vw"></center>
<br>
We’ve seen that $U$ and $V$ represent rotations, and $S$ represents scaling. Therefore, SVD says that any matrix can be decomposed into a rotation, then a scaling, and another rotation.
<center><img src = "images/svd_geo.png" width="600vw"></center> -->

### SVD in `NumPy`

For this demo, we'll continue working with our rectangular dataset from before with $n=100$ rows and $d=4$ columns.


In [74]:
# | code-fold: true
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(23)  # kallisti

plt.rcParams["figure.figsize"] = (4, 4)
plt.rcParams["figure.dpi"] = 150
sns.set()

rectangle = pd.read_csv("data/rectangle_data.csv")
rectangle.head(5)

,width,height,area,perimeter
0,8,6,48,28
1,2,4,8,12
2,1,3,3,8
3,9,3,27,24
4,9,8,72,34


In `NumPy`, the SVD decomposition function can be called with `np.linalg.svd` ([documentation](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)). There are multiple versions of SVD; to get the version that we will follow, we need to set the `full_matrices` parameter to `False`.


In [75]:
U, S, Vt = np.linalg.svd(rectangle, full_matrices=False)

First, let's examine `U`. As we can see, it's dimensions are $n \times d$.


In [76]:
U.shape

(100, 4)

The first 5 rows of `U` are shown below.


In [77]:
pd.DataFrame(U).head(5)

,0,1,2,3
0,-0.155151,0.064830,-0.029935,0.607042
1,-0.038370,-0.089155,0.062019,-0.608114
2,-0.020357,-0.081138,0.058997,-0.012249
3,-0.101519,-0.076203,-0.148160,0.045940
4,-0.218973,0.206423,0.007274,0.016538


$S$ is a little different in `NumPy`. Since the only useful values in the diagonal matrix $S$ are the singular values on the diagonal axis, only those values are returned and they are stored in an array.

Our `rectangle_data` has a rank of $3$, so we should have 3 non-zero singular values, **sorted from largest to smallest**.


In [78]:
S

array([3.62932568e+02, 6.29904732e+01, 2.56544651e+01, 4.96345451e-15])

It seems like we have 4 non-zero values instead of 3, but notice that the last value is so small ($10^{-15}$) that it's practically $0$. Hence, we can round the values to get 3 singular values.


In [79]:
np.round(S)

array([363.,  63.,  26.,   0.])

To get `S` in matrix format, we use `np.diag`.


In [80]:
Sm = np.diag(S)
Sm

array([[3.62932568e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.29904732e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.56544651e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.96345451e-15]])

Finally, we can see that `Vt` is indeed a $d \times d$ matrix.


In [81]:
Vt.shape

(4, 4)

In [82]:
pd.DataFrame(Vt)

,0,1,2,3
0,-0.146436,-0.129942,-8.100201e-01,-0.552756
1,-0.192736,-0.189128,5.863482e-01,-0.763727
2,-0.704957,0.709155,7.951614e-03,0.008396
3,-0.666667,-0.666667,-1.110223e-16,0.333333


To check that this SVD is a valid decomposition, we can reverse it and see if it matches our original table (it does, yay!).


In [83]:
pd.DataFrame(U @ Sm @ Vt).head(5)

,0,1,2,3
0,8.0,6.0,48.0,28.0
1,2.0,4.0,8.0,12.0
2,1.0,3.0,3.0,8.0
3,9.0,3.0,27.0,24.0
4,9.0,8.0,72.0,34.0


## PCA with SVD

Principal Component Analysis (PCA) and Singular Value Decomposition (SVD) can be easily mixed up, especially when you have to keep track of so many acronyms. Here is a quick summary:

- SVD: a linear algebra algorithm that splits a matrix into 3 component parts.
- PCA: a data science procedure used for dimensionality reduction that _uses_ SVD as one of the steps.

<!-- ### Derivation
::: {.callout-tip}
### [Linear Algebra Review] Covariance Matrix


::: -->

### Deriving Principal Components From SVD

After centering $X$ so that each column has a mean of 0, we find it's SVD:
$$ X = U S V^T $$

Because X is centered, the covariance matrix of X, $\Sigma$, is equal to $X^T X$. Rearranging this equation, we get

$$
\begin{align}
\Sigma &= X^T X \\
 &= (U S V^T)^T U S V^T \\
 &= V S^T U^T U S V^T & \text{U is orthonormal, so $U^T U = I$} \\
 &= V S^2 V^T
\end{align}
$$

Multiplying both sides by V, we get

$$
\begin{align}
\Sigma V &= VS^2 V^T V \\
&= V S^2
\end{align}
$$

This shows that The columns of V are the **eigenvectors** of the covariance matrix Σ and therefore the **principal components**. Additionally, the squared singular values $S^2$ are the **eigenvalues** of Σ.

<!-- <center><img src="images/slide15.png" alt='slide15' width='500'></center> -->

We've now shown that the first $k$ columns of $V$ (equivalently, the first $k$ rows of $V^{T}$) are the first k principal components of $X$. We can use them to construct the **latent vector representation** of $X$, $Z$, by projecting $X$ onto the principal components.

<!-- TODO if we have time: add lin alg review for projection  -->
<center><img src="images/Z.png" alt='slide16' width='500'></center>

From this equation $Z = X V$, we can compoute a second way to calculate $Z$:

$$
\begin{align}
Z &= X V \\
 &= USV^T V \\
 &= U S
\end{align}
$$

$$Z = XV = US$$

In other words, we can construct $X$'s' latent vector representation $Z$ through:

1. projecting $X$ onto the first $k$ columns of $V$, $V[:, :k]$
2. multiplying the first $k$ columns of U and the first $k$ rows of S

Using $Z$, we can approximately recover the original $X$ matrix by multiplying $Z$ by $V^T$:
$$ Z V^T = XV V^T = USV^T = X$$

### PCA Visualization

<center><img src="images/slide16.png" alt='slide16' width='500'></center>

The elements of each column of $V$ (row of $V^{T}$) rotate the original feature vectors into a principal component.

The first column of $V$ indicates how each feature contributes (e.g. positive, negative, etc.) to principal component 1.

<center><img src="images/rotate_center_plot.png" alt='slide17' width='750'></center>

Coupled together, this interpretation also allows us to understand that:

- The principal components are all **orthogonal** to each other because the columns of U are orthonormal.
- Principal components are **axis-aligned**. That is, if you plot two PCs on a 2D plane, one will lie on the x-axis, the other on the y-axis.
- Principal components are **linear combinations** of columns in our data X

### Using Principal Components

Let's summarize the steps to obtain Principal Components via SVD:

1. Center the data matrix by subtracting the mean of each attribute column.

2. To find the $k$ **principal components**:

   a. Compute the SVD of the data matrix ($X = U{S}V^{T}$)
   b. The first $k$ columns of $U{S}$ (or equivalently, $XV$) contain the $k$ **principal components** of $X$.

### Code Demo

In order to get the first $k$ principal components from an $n \times d$ matrix $X$, we:

1. Center $X$ by subtracting the mean from each column. Notice how we specify `axis=0` so that the mean is computed per column.


In [84]:
centered_df = rectangle - np.mean(rectangle, axis=0)
centered_df.head(5)

,width,height,area,perimeter
0,2.97,1.35,24.78,8.64
1,-3.03,-0.65,-15.22,-7.36
2,-4.03,-1.65,-20.22,-11.36
3,3.97,-1.65,3.78,4.64
4,3.97,3.35,48.78,14.64


2. Get the Singular Value Decomposition of centered $X$: $U$, $S$ and $V^T$


In [85]:
U, S, Vt = np.linalg.svd(centered_df, full_matrices=False)
Sm = pd.DataFrame(np.diag(np.round(S, 1)))

3. Multiply either $UΣ$ or $XV$. Mathematically, these give the same result, but computationally, floating point approximation results in slightly different numbers for very small values (check out the right-most column in the cells below).


In [86]:
# US
pd.DataFrame(U @ np.diag(S)).head(5)

,0,1,2,3
0,-26.432217,0.162686,0.807998,-2.278994e-15
1,17.045285,-2.181451,0.347732,4.388030e-15
2,23.245695,-3.538040,1.995334,-5.017992e-16
3,-5.383546,5.025395,0.253448,-9.949509e-17
4,-51.085217,-2.586948,2.099919,-3.425513e-16


In [87]:
# XV
pd.DataFrame(centered_df @ Vt.T).head(5)

,0,1,2,3
0,-26.432217,0.162686,0.807998,-2.664535e-15
1,17.045285,-2.181451,0.347732,1.332268e-15
2,23.245695,-3.538040,1.995334,2.220446e-15
3,-5.383546,5.025395,0.253448,-1.998401e-15
4,-51.085217,-2.586948,2.099919,-3.552714e-15


4. Take the first $k$ columns of $US$ (or $XV$). These are the first $k$ principal components of $X$.


In [88]:
two_PCs = (U @ np.diag(S))[:, :2]  # using US
two_PCs = (centered_df @ Vt.T).iloc[:, :2]  # using XV
pd.DataFrame(two_PCs).head()

,0,1
0,-26.432217,0.162686
1,17.045285,-2.181451
2,23.245695,-3.538040
3,-5.383546,5.025395
4,-51.085217,-2.586948


## Data Variance and Centering

We define the total variance of a data matrix as the sum of variances of attributes. The principal components are a low-dimension representation that capture as much of the original data's total variance as possible. Formally, the $i$-th singular value tells us the **component score**, i.e., how much of the data variance is captured by the $i$-th principal component. Supposing the number of datapoints is $n$:

$$\text{i-th component score} = \frac{(\text{i-th singular value}^2)}{n}$$

Summing up the component scores is equivalent to computing the total variance _if we center our data_.

**Data Centering**: PCA has a data centering step that precedes any singular value decomposition, where if implemented defines the component score as above.

If you want to dive deeper into PCA, [Steve Brunton's SVD Video Series](https://www.youtube.com/playlist?list=PLMrJAkhIeNNSVjnsviglFoY2nXildDCcv) is a great resource.

## Interpreting PCA

### PCA Plot

We often plot the first two principal components using a scatter plot, with PC1 on the $x$-axis and PC2 on the $y$-axis. This is often called a PCA plot.

If the first two singular values are large and all others are small, then two dimensions are enough to describe most of what distinguishes one observation from another. If not, then a PCA plot is omitting lots of information.

PCA plots help us assess similarities between our data points and if there are any clusters in our dataset. In the case study before, for example, we could create the following PCA plot:

<center><img src="images/pca_plot.png" alt='pca_plot' width='500'></center>

### Scree Plots

A scree plot shows the **variance ratio** captured by each principal component, with the largest variance ratio first. They help us visually determine the number of dimensions needed to describe the data reasonably. The singular values that fall in the region of the plot after a large drop-off correspond to principal components that are **not** needed to describe the data since they explain a relatively low proportion of the total variance of the data. This point where adding more principal components results in diminishing returns is called the "elbow” and is the point just before the line flattens out. Using this "elbow method", we can see that the elbow is at the second principal component.

<center><img src="images/scree_plot.png" alt='scree_plot' width='500'></center>

### Biplots

Biplots superimpose the directions onto the plot of PC2 vs PC1, where vector $j$ corresponds to the direction for feature $j$ (e.g. $v_{1j}, v_{2j}$). There are several ways to scale biplot vectors -- in this course, we plot the direction itself. For other scalings, which can lead to more interpretable directions/loadings, see [SAS biplots](https://blogs.sas.com/content/iml/2019/11/06/what-are-biplots.html)

Through biplots, we can interpret how features correlate with the principal components shown: positively, negatively, or not much at all.

<center><img src="images/slide17_2.png" alt='slide17_2' width='500'></center>

The directions of the arrow are ($v_1$, $v_2$) where $v_1$ and $v_2$ are how that specific feature column contributes to PC1 and PC2, respectively. $v_1$ and $v_2$ are elements of the first and second columns of $V$, respectively (i.e., the first two rows of $V^T$).

Say we were considering feature 3, and say that was the purple arrow labeled "520" here (pointing bottom right).

- $v_1$ and $v_2$ are the third elements of the respective columns in $V$. They are scale feature 3's column vector in the linear transformation to PC1 and PC2, respectively.
- Here we would infer that $v_1$ (in the $x$/PC1-direction) is positive, meaning that a linear increase in feature 3 would correspond to a linear increase of PC1, meaning feature 3 and PC1 are positively correlated.
- $v_2$ (in the $y$/pc2-direction) is negative, meaning a linear increase in feature 3 would result correspond to a linear decrease in PC2, meaning feature 3 and PC2 are negatively correlated.


## Example: House of Representatives Voting

Let's examine how the House of Representatives (of the 116th Congress, 1st session) voted in the month of September 2019.

Specifically, we’ll look at the records of Roll call votes. From the U.S. Senate ([link](https://www.senate.gov/reference/Index/Votes.htm)): Roll call votes occur when a representative or senator votes "yea" or "nay" so that the names of members voting on each side are recorded. A voice vote is a vote in which those in favor or against a measure say "yea" or "nay," respectively, without the names or tallies of members voting on each side being recorded.

**Do legislators' roll call votes show a relationship with their political party?**


In [89]:
# | code-fold: true
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yaml
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go


votes = pd.read_csv("data/votes.csv")
votes = votes.astype({"roll call": str})
votes.head()

,chamber,session,roll call,member,vote
0,House,1,555,A000374,Not Voting
1,House,1,555,A000370,Yes
2,House,1,555,A000055,No
3,House,1,555,A000371,Yes
4,House,1,555,A000372,No


Suppose we pivot this table to group each legislator and their voting pattern across every (roll call) vote in this month. We mark 1 if the legislator voted Yes (yea), and 0 otherwise (No/nay, no vote, speaker, etc.).


In [90]:
# | code-fold: true
def was_yes(s):
    return 1 if s.iloc[0] == "Yes" else 0


vote_pivot = votes.pivot_table(
    index="member", columns="roll call", values="vote", aggfunc=was_yes, fill_value=0
)
print(vote_pivot.shape)
vote_pivot.head()

(441, 41)


roll call,515,516,517,518,519,520,521,522,523,524,...,546,547,548,549,550,551,552,553,554,555
member,,,,,,,,,,,,,,,,,,,,,
A000055,1,0,0,0,1,1,0,1,1,1,...,0,0,1,0,0,1,0,0,1,0
A000367,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,1,0,1
A000369,1,1,0,0,1,1,0,1,1,1,...,0,0,1,0,0,1,0,0,1,0
A000370,1,1,1,1,1,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,1
A000371,1,1,1,1,1,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,1


How do we analyze this data?

While we could consider loading information about the legislator, such as their party, and see how this relates to their voting pattern, it turns out that we can do a lot with PCA to cluster legislators by how they vote.

### PCA


In [91]:
vote_pivot_centered = vote_pivot - np.mean(vote_pivot, axis=0)
u, s, vt = np.linalg.svd(vote_pivot_centered, full_matrices=False)

### PCA Plot


In [92]:
import plotly.io as pio

# Automatically select the appropriate renderer for your environment
pio.renderers.default = "notebook_connected"

vote_2d = pd.DataFrame(index=vote_pivot_centered.index)
vote_2d[["z1", "z2", "z3"]] = (u * s)[:, :3]
fig = px.scatter(vote_2d, x="z1", y="z2", title="Vote Data", width=800, height=600)
fig.show()

It would be interesting to see the political affiliation for each vote.

### Component Scores

If the first two singular values are large and all others are small, then two dimensions are enough to describe most of what distinguishes one observation from another. If not, then a PCA scatter plot is omitting lots of information.

An equivalent way to evaluate this is to determine the **variance ratios**, i.e., the fraction of the variance each PC contributes to total variance.


In [93]:
np.round(s**2 / sum(s**2), 2)

array([0.8 , 0.05, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

### Scree plot


In [94]:
fig = px.line(
    y=s**2 / sum(s**2), title="Variance Explained", width=700, height=600, markers=True
)
fig.update_xaxes(title_text="Principal Component")
fig.update_yaxes(title_text="Proportion of Variance Explained")

fig.show()

## Incorporating Member Information

Suppose we load in more member information, from https://github.com/unitedstates/congress-legislators. This includes each legislator's political party.


In [95]:
# | code-fold: true

legislators_data = yaml.safe_load(open("data/legislators-2019.yaml"))


def to_date(s):
    return datetime.strptime(s, "%Y-%m-%d")


legs = pd.DataFrame(
    columns=[
        "leg_id",
        "first",
        "last",
        "gender",
        "state",
        "chamber",
        "party",
        "birthday",
    ],
    data=[
        [
            x["id"]["bioguide"],
            x["name"]["first"],
            x["name"]["last"],
            x["bio"]["gender"],
            x["terms"][-1]["state"],
            x["terms"][-1]["type"],
            x["terms"][-1]["party"],
            to_date(x["bio"]["birthday"]),
        ]
        for x in legislators_data
    ],
)
legs["age"] = 2024 - legs["birthday"].dt.year
legs.set_index("leg_id")
legs.sort_index()

vote_2d = vote_2d.join(legs.set_index("leg_id")).dropna()

np.random.seed(42)
vote_2d["z1_jittered"] = vote_2d["z1"] + np.random.normal(0, 0.1, len(vote_2d))
vote_2d["z2_jittered"] = vote_2d["z2"] + np.random.normal(0, 0.1, len(vote_2d))
vote_2d["z3_jittered"] = vote_2d["z3"] + np.random.normal(0, 0.1, len(vote_2d))

fig = px.scatter(
    vote_2d,
    x="z1_jittered",
    y="z2_jittered",
    color="party",
    symbol="gender",
    size="age",
    title="Vote Data",
    width=800,
    height=600,
    size_max=10,
    opacity=0.7,
    color_discrete_map={
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "green",
    },
    hover_data=["first", "last", "state", "party", "gender", "age"],
)

fig.show()

### Analysis: Regular Voters

Not everyone voted all the time. Let's examine the frequency of voting.
First, let's recompute the pivot table where we only consider Yes/No votes, and ignore records with "No Vote" or other entries.


In [96]:
# | code-fold: true

vote_2d["num votes"] = votes[votes["vote"].isin(["Yes", "No"])].groupby("member").size()
vote_2d.dropna(inplace=True)
vote_2d.head()

,z1,z2,z3,first,last,gender,state,chamber,party,birthday,age,z1_jittered,z2_jittered,z3_jittered,num votes
member,,,,,,,,,,,,,,,
A000055,3.061356,0.364191,0.194493,Robert,Aderholt,M,AL,rep,Republican,1965-07-22,59.0,3.111028,0.358637,0.192851,40.0
A000367,0.188870,-2.433565,0.283280,Justin,Amash,M,MI,rep,Independent,1980-04-18,44.0,0.175043,-2.395158,0.402119,41.0
A000369,2.844370,0.821619,-0.476126,Mark,Amodei,M,NV,rep,Republican,1958-06-12,66.0,2.909139,0.818350,-0.223433,41.0
A000370,-2.607536,0.127977,0.031377,Alma,Adams,F,NC,rep,Democrat,1946-05-27,78.0,-2.455233,-0.078767,-0.021710,41.0
A000371,-2.607536,0.127977,0.031377,Pete,Aguilar,M,CA,rep,Democrat,1979-06-19,45.0,-2.630951,0.119065,-0.017567,41.0


### Exploring the Principal Components

We can also look at Vt directly to try to gain insight into why each component is as it is.


In [97]:
# | code-fold: true

fig_eig = px.bar(x=vote_pivot_centered.columns, y=vt[0, :])
# extract the trace from the figure
fig_eig.show()

We have the party affiliation labels so we can see if this eigenvector aligns with one of the parties.


In [98]:
# | code-fold: true

party_line_votes = (
    vote_pivot_centered.join(legs.set_index("leg_id")["party"])
    .groupby("party")
    .mean()
    .T.reset_index()
    .rename(columns={"index": "call"})
    .melt("call")
)
fig = px.bar(
    party_line_votes,
    x="call",
    y="value",
    facet_row="party",
    color="party",
    color_discrete_map={
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "green",
    },
)
fig_each = fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_each.show()

### Biplot


In [99]:
# | code-fold: true

loadings = pd.DataFrame(
    {"pc1": np.sqrt(s[0]) * vt[0, :], "pc2": np.sqrt(s[1]) * vt[1, :]},
    index=vote_pivot_centered.columns,
)
fig = px.scatter(
    vote_2d,
    x="z1_jittered",
    y="z2_jittered",
    color="party",
    symbol="gender",
    size="num votes",
    title="Biplot",
    width=800,
    height=600,
    size_max=10,
    opacity=0.7,
    color_discrete_map={
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "green",
    },
    hover_data=["first", "last", "state", "party", "gender", "age"],
)

for call, pc1, pc2 in loadings.head(20).itertuples():
    fig.add_scatter(
        x=[0, pc1],
        y=[0, pc2],
        name=call,
        mode="lines+markers",
        textposition="top right",
        marker=dict(size=10, symbol="arrow-bar-up", angleref="previous"),
    )
fig.show()

Each roll call from the 116th Congress - 1st Session: https://clerk.house.gov/evs/2019/ROLL_500.asp

- 555: Raising a question of the privileges of the House ([H.Res.590](https://www.congress.gov/bill/116th-congress/house-resolution/590))
- 553: [https://www.congress.gov/bill/116th-congress/senate-joint-resolution/54/actions]
- 527: On Agreeing to the Amendment [H.R.1146 - Arctic Cultural and Coastal Plain Protection Act](https://www.congress.gov/bill/116th-congress/house-bill/1146)


Please visit this [link](https://data100.datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FDS-100%2Ffa23-student&urlpath=lab%2Ftree%2Ffa23-student%2Flecture%2Flec26%2Flec26-votes.ipynb&branch=main) to see the full Jupyter notebook demo.

As shown in the demo, the primary goal of PCA is to transform observations from high-dimensional data down to low dimensions through linear transformations.

A related goal of PCA connects back to the idea that a low-dimension representation of the data should capture the variability of the original data. For example, if the first two singular values are large and the others are relatively small, then two dimensions are probably enough to describe most of what distinguishes one observation from another. However, if this is not the case, then a PCA scatter plot is probably omitting lots of information.

We can use the following formulas to quantify the amount each principal component contributes to the total variance:

$$ \text{component score} = \frac{s_i^{2}}{N}$$

$$ \text{total variance} = \text{sum of all the component scores} = \sum\_{i=1}^k \frac{s_i^{2}}{N} $$

$$ \text{variance ratio of principal component i} = \frac{\text{component score i}}{\text{total variance}} = \frac{s*i^{2} / N}{\sum*{i=1}^k s_i^{2} / N}$$

In Python, assuming you had a 1D `NumPy` array of singular values `s` returned by `np.linalg.svd`, you could compute the list of variances ratios with `s**2 / sum(s**2)`.


## Example: Image Classification

In machine learning, PCA is often used as a preprocessing step prior to training a supervised model.

See the following [demo](https://data100.datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FDS-100%2Ffa23-student&urlpath=lab%2Ftree%2Ffa23-student%2Flecture%2Flec26%2Flec26-fashion-mnist.ipynb&branch=main) to see how PCA is useful for building an image classification model based on the MNIST-Fashion dataset.

<center><img src="images/mnist.png" alt='slide21' width='500'></center>


In [36]:
# | code-fold: true

import requests
from pathlib import Path
import time


def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.

    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded

    return: The pathlib.Path object representing the file.
    """

    ### BEGIN SOLUTION
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print("Downloading...", end=" ")
        resp = requests.get(data_url)
        with file_path.open("wb") as f:
            f.write(resp.content)
        print("Done!")
        last_modified_time = time.ctime(file_path.stat().st_mtime)
    else:
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version that was downloaded (UTC):", last_modified_time)
    return file_path
    ### END SOLUTION


def head(filename, lines=5):
    """
    Returns the first few lines of a file.

    filename: the name of the file to open
    lines: the number of lines to include

    return: A list of the first few lines from the file.
    """
    from itertools import islice

    with open(filename, "r") as f:
        return list(islice(f, lines))


import gzip
import os

import numpy as np


def load_data():
    """Loads the Fashion-MNIST dataset.

    This is a dataset of 60,000 28x28 grayscale images of 10 fashion categories,
    along with a test set of 10,000 images. This dataset can be used as
    a drop-in replacement for MNIST.

    The classes are:

    | Label | Description |
    |:-----:|-------------|
    |   0   | T-shirt/top |
    |   1   | Trouser     |
    |   2   | Pullover    |
    |   3   | Dress       |
    |   4   | Coat        |
    |   5   | Sandal      |
    |   6   | Shirt       |
    |   7   | Sneaker     |
    |   8   | Bag         |
    |   9   | Ankle boot  |

    Returns:
      Tuple of NumPy arrays: `(x_train, y_train), (x_test, y_test)`.

    **x_train**: uint8 NumPy array of grayscale image data with shapes
      `(60000, 28, 28)`, containing the training data.

    **y_train**: uint8 NumPy array of labels (integers in range 0-9)
      with shape `(60000,)` for the training data.

    **x_test**: uint8 NumPy array of grayscale image data with shapes
      (10000, 28, 28), containing the test data.

    **y_test**: uint8 NumPy array of labels (integers in range 0-9)
      with shape `(10000,)` for the test data.

    Example:

    ```python
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    assert x_train.shape == (60000, 28, 28)
    assert x_test.shape == (10000, 28, 28)
    assert y_train.shape == (60000,)
    assert y_test.shape == (10000,)
    ```

    License:
      The copyright for Fashion-MNIST is held by Zalando SE.
      Fashion-MNIST is licensed under the [MIT license](
      https://github.com/zalandoresearch/fashion-mnist/blob/master/LICENSE).

    """
    dirname = os.path.join("datasets", "fashion-mnist")
    base = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/"
    files = [
        "train-labels-idx1-ubyte.gz",
        "train-images-idx3-ubyte.gz",
        "t10k-labels-idx1-ubyte.gz",
        "t10k-images-idx3-ubyte.gz",
    ]

    paths = []
    for fname in files:
        paths.append(fetch_and_cache(base + fname, fname))
        # paths.append(get_file(fname, origin=base + fname, cache_subdir=dirname))

    with gzip.open(paths[0], "rb") as lbpath:
        y_train = np.frombuffer(lbpath.read(), np.uint8, offset=8)

    with gzip.open(paths[1], "rb") as imgpath:
        x_train = np.frombuffer(imgpath.read(), np.uint8, offset=16).reshape(
            len(y_train), 28, 28
        )

    with gzip.open(paths[2], "rb") as lbpath:
        y_test = np.frombuffer(lbpath.read(), np.uint8, offset=8)

    with gzip.open(paths[3], "rb") as imgpath:
        x_test = np.frombuffer(imgpath.read(), np.uint8, offset=16).reshape(
            len(y_test), 28, 28
        )

    return (x_train, y_train), (x_test, y_test)

In [37]:
# | code-fold: true


(train_images, train_labels), (test_images, test_labels) = load_data()
print("Training images", train_images.shape)
print("Test images", test_images.shape)

Downloading... Done!
Downloading... Done!
Downloading... Done!
Downloading... Done!
Training images (60000, 28, 28)
Test images (10000, 28, 28)


We have loaded a lot of data which you can play with later (try building a classifier).
For the purposes of this demo, let's take a small sample of the training data.


In [39]:
# | code-fold: true

class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
class_dict = {i: class_name for i, class_name in enumerate(class_names)}

rng = np.random.default_rng(42)
n = 5000
sample_idx = rng.choice(np.arange(len(train_images)), size=n, replace=False)

# Invert and normalize the images so they look better
img_mat = -1 * train_images[sample_idx]
img_mat = (img_mat - img_mat.min()) / (img_mat.max() - img_mat.min())

images = pd.DataFrame(
    {
        "images": img_mat.tolist(),
        "labels": train_labels[sample_idx],
        "class": [class_dict[x] for x in train_labels[sample_idx]],
    }
)
images.head()

,images,labels,class
0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",3,Dress
1,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",4,Coat
2,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0,T-shirt/top
3,"[[1.0, 1.0, 1.0, 1.0, 1.0, 0.996078431372549, ...",2,Pullover
4,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",1,Trouser


### Visualizing images


In [101]:
def show_images(images, ncols=5, max_images=30):
    # conver the subset of images into a n,28,28 matrix for facet visualization
    img_mat = np.array(images.head(max_images)["images"].to_list())
    fig = px.imshow(
        img_mat,
        color_continuous_scale="gray",
        facet_col=0,
        facet_col_wrap=ncols,
        height=220 * int(np.ceil(len(images) / ncols)),
    )
    fig.update_layout(coloraxis_showscale=False)
    # Extract the facet number and convert it back to the class label.
    fig.for_each_annotation(
        lambda a: a.update(text=images.iloc[int(a.text.split("=")[-1])]["class"])
    )
    return fig


fig = show_images(images.groupby("class", as_index=False).sample(2), ncols=6)
fig.show()

### PCA

How would we visualize the entire dataset? Let's use PCA to find a low dimensional representation of the images.

First, let's understand the high-dimensional representation. We will extract the matrix of images from the dataframe:


In [41]:
X = np.array(images["images"].to_list())

X = X.reshape(X.shape[0], -1)

X = X - X.mean(axis=0)

from sklearn.decomposition import PCA

n_comps = 50
pca = PCA(n_components=n_comps)
pca.fit(X)

PCA(n_components=50)

### Examining PCA Results


In [100]:
# make a line plot and show markers
fig = px.line(y=pca.explained_variance_ratio_ * 100, markers=True)
fig.show()

Most of data is explained in first two or three dimensions.


<br>
The demo shows how we can use PCA during the Exploratory Data Analysis stage of our data science lifecycle to:
- visually identify clusters of similar observations in high dimensions.
- find a small basis for representing variations in complex things.
- reduce the number of dimensions to make some computations cheaper.


## (Bonus) Applications of PCA

### PCA in Biology

PCA is commonly used in biomedical contexts, which have many named variables!

1. To cluster data ([Paper 1](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2680-1), [Paper 2](https://www.science.org/doi/10.1126/scirobotics.abk2378))
2. To identify correlated variables ([interpret](https://docs.google.com/presentation/d/1-aDu0ILCkPx3iCcJGB3YXci-L4g90Q6AarXU6wffLB8/edit#slide=id.g62cb86badb_0_1128) rows of $V^{T}$ as linear coefficients) ([Paper 3](https://www.nature.com/articles/s41598-017-05714-1)). Uses [biplots](<https://www.google.com/url?q=https://www.geo.fu-berlin.de/en/v/soga/Geodata-analysis/Principal-Component-Analysis/principal-components-basics/Interpretation-and-visualization/index.html%23:~:text%3DThe%2520biplot%2520is%2520a%2520very,in%2520a%2520single%2520biplot%2520display.%26text%3DThe%2520plot%2520shows%2520the%2520observations,principal%2520components%2520(synthetic%2520variables).&sa=D&source=editors&ust=1682131633152964&usg=AOvVaw2H9SOeMP5kUS890Fkhfthx>).

### Why Perform PCA

We often perform PCA during the Exploratory Data Analysis (EDA) stage of our data science lifecycle (if we already know what to model, we probably don't need PCA); it helps us with:

- Visually identifying clusters of similar observations in high dimensions.
- Removing irrelevant dimensions if we suspect that the dataset is inherently low rank. For example, if the columns are collinear: there are many attributes but only a few mostly determine the rest through linear associations.
- Finding a small basis for representing variations in complex things, e.g., images, genes.
- Reducing the number of dimensions to make some computation cheaper.

### Why PCA, then Model?

1. Reduces dimensionality, allowing us to speed up training and reduce the number of features, etc.
2. Avoids multicollinearity in the new features created (i.e. the principal components)

<center><img src="images/slide21.png" alt='slide21' width='500'></center>


## (Bonus) PCA vs. Regression

### Regression: Minimizing Horizontal/Verticle Error

Suppose we know the child mortality rate of a given country. Linear regression tries to predict the fertility rate from the mortality rate; for example, if the mortality is 6, we might guess the fertility is near 4. The regression line tells us the “best” prediction of fertility given all possible mortality values by minimizing the root mean squared error [see vertical red lines, only some shown].

<center><img src = "images/lin_reg.png" width="400vw"></center>
<br>
We can also perform a regression in the reverse direction, that is, given the fertility, we try to predict the mortality. In this case, we get a different regression line which minimizes the root mean squared length of the horizontal lines.

<center><img src = "images/lin_reg_reverse.png" width="400vw"></center>

### SVD: Minimizing Perpendicular Error

The rank 1 approximation is close but not the same as the mortality regression line. Instead of minimizing _horizontal_ or _vertical_ error, our rank 1 approximation minimizes the error _perpendicular_ to the subspace onto which we’re projecting. That is, SVD finds the line such that if we project our data onto that line, the error between the projection and our original data is minimized. The similarity of the rank 1 approximation and the fertility was just a coincidence. Looking at adiposity and bicep size from our body measurements dataset, we see the 1D subspace onto which we are projecting is between the two regression lines.

<center><img src = "images/rank1.png" width="400vw"></center>

### Beyond 1D and 2D

In higher dimensions, the idea behind principal components is just the same! Suppose we have 30-dimensional data and decide to use the first 5 principal components. Our procedure minimizes the error between the original 30-dimensional data and the projection of that 30-dimensional data on to the “best” 5-dimensional subspace. See [CS189](https://www.eecs189.org/static/notes/n10.pdf) for more details.


## (Bonus) Automatic Factorization

One key fact to remember is that the decomposition are not arbitrary. The _rank_ of a matrix limits how small our inner dimensions can be if we want to perfectly recreate our matrix. The proof for this is out of scope.

Even if we know we have to factorize our matrix using an inner dimension of R, that still leaves a large space of solutions to traverse. What if we have a procedure to automatically factorize a rank R matrix into an R dimensional representation with some transformation matrix?

- Lower dimensional representation avoids redundant features.
- Imagine a 1000-dimensional dataset: If the rank is only 5, it’s much easier to do EDA after this mystery procedure.

What if we wanted a 2D representation? It's valuable to compress all of the data that is relevant onto as few dimensions as possible in order to plot it efficiently. Some 2D matrices yield better approximations than others. How well can we do?


## (Bonus) Proof of Component Score

The proof defining component score is out of scope for this class, but it is included below for your convenience.

_Setup_: Consider the design matrix $X \in \mathbb{R}^{n \times d}$, where the $j$-th column (corresponding to the $j$-th feature) is $x_j \in \mathbb{R}^n$ and the element in row $i$, column $j$ is $x_{ij}$. Further, define $\tilde{X}$ as the **centered** design matrix. The $j$-th column is $\tilde{x}_j \in \mathbb{R}^n$ and the element in row $i$, column $j$ is $\tilde{x}_{ij} = x_{ij} - \bar{x_j}$, where $\bar{x_j}$ is the mean of the $x_j$ column vector from the original $X$.

_Variance_: Construct the **covariance matrix**: $\frac{1}{n} \tilde{X}^T \tilde{X} \in \mathbb{R}^{d \times d}$. The $j$-th element along the diagonal is the **variance** of the $j$-th column of the original design matrix $X$:

$$\left( \frac{1}{n} \tilde{X}^T \tilde{X} \right)_{jj} = \frac{1}{n} \tilde{x}_j ^T \tilde{x}_j = \frac{1}{n} \sum_{i=i}^n (\tilde{x}_{ij} )^2 = \frac{1}{n} \sum_{i=i}^n (x_{ij} - \bar{x_j})^2$$

_SVD_: Suppose singular value decomposition of the _centered_ design matrix $\tilde{X}$ yields $\tilde{X} = U S V^T$, where $U \in \mathbb{R}^{n \times d}$ and $V \in \mathbb{R}^{d \times d}$ are matrices with orthonormal columns, and $S \in \mathbb{R}^{d \times d}$ is a diagonal matrix with singular values of $\tilde{X}$.

$$
\begin{aligned}
\tilde{X}^T \tilde{X} &= (U S V^T )^T (U S V^T) \\
&= V S U^T U S V^T  & (S^T = S) \\
&= V S^2 V^T & (U^T U = I) \\
\frac{1}{n} \tilde{X}^T \tilde{X} &= \frac{1}{n} V S V^T =V \left( \frac{1}{n} S \right) V^T \\
\frac{1}{n} \tilde{X}^T \tilde{X} V &= V \left( \frac{1}{n} S \right) V^T V = V \left( \frac{1}{n} S \right) & \text{(right multiply by }V \rightarrow V^T V = I \text{)} \\
V^T \frac{1}{n} \tilde{X}^T \tilde{X} V &= V^T V \left( \frac{1}{n} S \right) = \frac{1}{n} S & \text{(left multiply by }V^T \rightarrow V^T V = I \text{)} \\
\left( \frac{1}{n} \tilde{X}^T \tilde{X} \right)_{jj} &= \frac{1}{n}S_j^2  & \text{(Define }S_j\text{ as the} j\text{-th singular value)} \\
\frac{1}{n} S_j^2 &= \frac{1}{n} \sum_{i=i}^n (x_{ij} - \bar{x_j})^2
\end{aligned}
$$

The last line defines the $j$-th component score.
